In [1]:
import torch

from torchvision.models import mobilenet_v3_large, mobilenet_v3_small, MobileNet_V3_Large_Weights, MobileNet_V3_Small_Weights
from torchvision import transforms

from PIL import Image
from nets.nn import mobilenet_v3_large as m3l
from nets.nn import mobilenet_v3_small as m3s
from assets.meta import IMAGENET_CATEGORIES

In [2]:
mobilenet_v3_l = mobilenet_v3_large(weights=MobileNet_V3_Large_Weights.IMAGENET1K_V1)
mobilenet_v3_s = mobilenet_v3_small(weights=MobileNet_V3_Small_Weights.IMAGENET1K_V1)

In [3]:
x = lambda model: sum(p.numel() for p in model.parameters() if p.requires_grad)
x(mobilenet_v3_l)

5483032

In [4]:
x(mobilenet_v3_s)

2542856

In [5]:
def preprocess_image(image_path):
    transform = transforms.Compose([
        transforms.Resize((224, 224)),  # Resize the image to match the model's input size
        transforms.ToTensor(),  # Convert the image to a PyTorch tensor
        transforms.Normalize(
            mean=[0.485, 0.456, 0.406],  # Normalize the image using the mean and std of ImageNet
            std=[0.229, 0.224, 0.225]
        ),
    ])

    image = Image.open(image_path)
    image = transform(image).unsqueeze(0)  # Add a batch dimension
    return image

In [6]:
def inference(model, image_path):
    model.eval()

    input_image = preprocess_image(image_path)
    with torch.no_grad():
        output = model(input_image)

    _, predicted_class = output.max(1)
    print(f"Predicted class index: {predicted_class.item()}")

    predicted_label = IMAGENET_CATEGORIES[predicted_class.item()]
    print(f"Predicted class label: {predicted_label}")

In [7]:
inference(mobilenet_v3_s, "assets/tabby_cat.jpg")

Predicted class index: 281
Predicted class label: tabby


In [8]:
local_m3l = m3l()
local_m3l.load_state_dict(mobilenet_v3_l.state_dict())

<All keys matched successfully>

In [9]:
inference(local_m3l, "assets/tabby_cat.jpg")

Predicted class index: 281
Predicted class label: tabby


In [10]:
x(local_m3l)

5483032

In [11]:
local_m3s = m3s()
local_m3s.load_state_dict(mobilenet_v3_s.state_dict())

<All keys matched successfully>

In [12]:
inference(local_m3s, "assets/tabby_cat.jpg")

Predicted class index: 281
Predicted class label: tabby


In [13]:
torch.save(local_m3s.half().state_dict(), "./weights/mobilenet_v3_small_half.pt")
torch.save(local_m3l.half().state_dict(), "./weights/mobilenet_v3_large_half.pt")
